# IBM Applied Data Science Capstone Course - Santa Clara Travellers Guide.
### Final Project Submission
- Build a dataframe of the postal codes for Santa Clara. Find businesses in the area, generate a rating for the businesses by getting the data from yelp and also retrieve traffic incident patterns. Build an analysis of which postal codes are better for living and for which ones to visit.
***
### 1. Build a dataset of postal codes with a boundbox
- Use FourSquare API to find venues near Santa Clara, CA. Build a dataframe of the postal codes for Santa Clara. Find venues in that area. 

In [1]:
import pandas as pd # library for data analsysis

import math

import re

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium # map rendering library

from IPython.display import display #To print dataframe as html table

import numpy as np # library to handle data in a vectorized manner

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Idea is in radius of 10kms find venues for Santa Clara, CA
radius = 10000
LIMIT = 1000
FOURSQUARE_API_KEY= 'KDCT4XDGGZLR4H04L2JWP355LPPEEI3SEQSD3BRKPDDLIW55'
FOURSQUARE_SECRET = 'QFJU1DJ4FRLRGCR0HNAWN151LPYX02MDIQL3A0SNKPNDACOS'
FOURSQUARE_VERSION = '20210730'
NEAR_LOCATION = 'Santa Clara, CA'


BING_API_KEY= "&o=json&key=AlduvugCPyKB2jO4Sq95h_u4UVyFQ62cNp60YW8yDS97RHbsZ5K5aHlhs0g1GlWw"
YELP_API_KEY= "Igwqf1HivnhqBA6oP1oRFF8tqHZgUSdtM6rmeMl0uBxSsyMyJrDN3Dja954KwhazNvJf8CZr3hG6fLMJDZyTqvfazmfxmveHCQbqcGWWhtBNA5-GjTFmuN_6g8EKYXYx" 

yelp_url = 'https://api.yelp.com/v3/businesses/search'
yelp_headers = {'Authorization': 'Bearer {}'.format(YELP_API_KEY)}
incident_url = 'https://dev.virtualearth.net/REST/v1/Traffic/Incidents/'
    


#Getting all the venues
venues = []

url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}".format(
        FOURSQUARE_API_KEY,
        FOURSQUARE_SECRET,
        FOURSQUARE_VERSION,
        NEAR_LOCATION,
        radius, 
        LIMIT)

#print(requests.get(url).json())

results = requests.get(url).json()["response"]['groups'][0]['items']
#results = requests.get(url).json()["response"]['venues']
#print(results)

venue_name=""
venue_lat=1000.00000001
venue_long=1000.00000001
venue_zip="80000"
venue_cat_name=""
venue_street=""

for venue in results:
        try:
            venue_name=venue['venue']['name']
        except:
            venue_name="NA"
        try:
            venue_lat=venue['venue']['location']['lat']
        except:
            venue_lat=0.0
        try:
            venue_long=venue['venue']['location']['lng']
        except:
            venue_long=0.0
        try:
            venue_zip=venue['venue']['location']['postalCode']
        except:
            venue_zip="NA"
        try:
            venue_cat_name=venue['venue']['categories'][0]['name']
        except:
            venue_cat_name="NA"
        try:
            venue_street=venue['venue']['location']['formattedAddress'][0]
            if venue_street.find('(') > 0:   #There is a paran in the name of the street name
                venue_street=venue_street[venue_street.find(' '):venue_street.find('(')]
            else:
                venue_street=venue_street[venue_street.find(' '):]
            
        except:
            venue_cat_name="NA"
            
        #print(venue['venue']['location']['postalCode'])
        venues.append((
            venue_zip,
            venue_name,
            venue_street,
            venue_cat_name,
            venue_lat, 
            venue_long))
        
venues_df = pd.DataFrame(venues)


#print(venues_df.shape)

# define the column names
venues_df.columns = ['PostalCode', 'Name', 'Street', 'Category Name', 'Latitude', 'Longitude']

#df_result = venues_df.groupby('PostalCode').agg({'Latitude':'min', 'Longitude':'max'}).reset_index()

#df_result = df_result[df_result['PostalCode'] != 'NA']

#df_result['BoundBox']=df_result.apply(lambda x:'%s, %s' % (x['Latitude'],x['Longitude']),axis=1)

#del(df_result['Latitude'])
#del(df_result['Longitude'])

#df_result=pd.merge(venues_df,df_result,on='PostalCode')

#If the postalcode is not available then remove the boundbox
#df_result['BoundBox'] = df_result.apply(lambda x: '' if x['PostalCode']=='NA' else x['PostalCode'], axis=1)

#Filter out values which have postalcode=NA
#df_result.loc[df_result['PostalCode'] == 'NA']


#determine the row indexes for the NA postalcode values
row_indices = venues_df[ venues_df['PostalCode'] == 'NA' ].index

venues_df.drop(row_indices, inplace = True)

display(venues_df)


,PostalCode,Name,Street,Category Name,Latitude,Longitude
0,95050,Poke Poke Fish Bar,El Camino,Seafood Restaurant,37.352162,-121.967648
1,95050,Sprouts Farmers Market,El Camino Real,Grocery Store,37.350087,-121.962339
2,95050,Dusita Thai,El Camino Real Ste 104105,Thai Restaurant,37.352780,-121.965524
3,95050,Mission Santa Clara de Asís,El Camino Real,Church,37.349250,-121.941514
4,95110,Airfield Supply Co,Coleman Ave,Marijuana Dispensary,37.356900,-121.931689
...,...,...,...,...,...,...
95,95113,Original Gravity Public House,S 1st St,Pub,37.335052,-121.889747
96,94086,Braly Park,Daffodil Ct.,Park,37.364469,-122.019604
97,95110,Market Beer Company,Pedro Square,Pub,37.336333,-121.895059
98,95131,California Sourdough,Murphy Ave Ste A,Sandwich Place,37.382188,-121.895235


### 2. Define a method to calculate a boundbox for given latitude and longitude
- To generate a boundbox for a given latitude and longitude within an area of 5 kms around the business. 

In [2]:
# degrees to radians
def deg2rad(degrees):
    return math.pi*degrees/180.0
# radians to degrees
def rad2deg(radians):
    return 180.0*radians/math.pi

# Semi-axes of WGS-84 geoidal reference
WGS84_a = 6378137.0  # Major semiaxis [m]
WGS84_b = 6356752.3  # Minor semiaxis [m]

# Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
def WGS84EarthRadius(lat):
    # http://en.wikipedia.org/wiki/Earth_radius
    An = WGS84_a*WGS84_a * math.cos(lat)
    Bn = WGS84_b*WGS84_b * math.sin(lat)
    Ad = WGS84_a * math.cos(lat)
    Bd = WGS84_b * math.sin(lat)
    return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def boundingBox(latitudeInDegrees, longitudeInDegrees, halfSideInKm):
    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    halfSide = 1000*halfSideInKm

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - halfSide/radius
    latMax = lat + halfSide/radius
    lonMin = lon - halfSide/pradius
    lonMax = lon + halfSide/pradius

    return (str(round(rad2deg(latMin),3))+","+str(round(rad2deg(lonMin),3))+","+str(round(rad2deg(latMax),3))+","+str(round(rad2deg(lonMax),3)))

### 3. Define a method to get the details such as rating of the business with help of Yelp API and get BING data of traffic incidents within 5 kms of the same business based on a boundbox set of co-ordinates.
- Ratings from Yelp are used to grade the business and the traffic incidents in the area of the business are retrieved from BING to help determine the congestion.
- The category list for yelp is left open and search is based on "term" as business name.

In [3]:
def get_yelp_bing_data(latitude,longitude,biz_name,cat_name,street_name):
    title=""
    zipcode=[]
    bizname=[]
    bizrating=[]
    bizadd=[]
    biz_lat=[]
    biz_long=[]
    biz_title=[]
    bound_box=[]
    traffic_incidents=[]
    road_closures=[]
    bound_box_str=""
    estimatetotalincidents=0
    roadclosures=0
    response_data=[]
    category_name=[]
    street_nm=[]
    
    yelp_url_params = {
	'latitude': latitude, 
	'longitude': longitude,
	'term': biz_name,
	'limit': 1
	}
    
    #print(biz_name+" - ")
    #print(yelp_url_params)
    response_data = requests.get(yelp_url, headers=yelp_headers, params=yelp_url_params)
    
    #print (response_data.json())
    
    #for business in response_parks.json()["businesses"]:
    
    if response_data:
        #print("Getting Yelp and Bing Data for : "+biz_name+" at "+str(latitude)+","+str(longitude))
        #print(response_data.json())
        for business in response_data.json()["businesses"]:
            if any(s in business["name"] for s in biz_name):
                title=""
                incident_url="https://dev.virtualearth.net/REST/v1/Traffic/Incidents/"
                roadclosures=0
                for i in range(0,len(business["categories"])):
                    if title == "":
                        title=business["categories"][i].get('title')
                    else:
                        title=title+" , "+business["categories"][i].get('title')
                zipcode.append(business["location"]["zip_code"])
                bizname.append(business["name"])
                bizrating.append(business["rating"])
                bizadd.append(business["location"]["display_address"][0])
                biz_lat.append(business["coordinates"]["latitude"])
                biz_long.append(business["coordinates"]["longitude"])
                biz_title.append(title)
                category_name.append(cat_name)
                street_nm.append(street_name)
                bound_box_str=boundingBox(float(business["coordinates"]["latitude"]),float(business["coordinates"]["longitude"]),5)
                bound_box.append(bound_box_str)
                incident_url = incident_url+bound_box_str+"/true?t=1,2,3,4,5,9&s=3,4"+BING_API_KEY;
                #print(incident_url)
                incident_response=requests.request('GET', incident_url)
                #print("Yelp Processing:"+biz_name+" at "+business["location"]["zip_code"])
                #print(incident_response.json())
                estimatetotalincidents=int(incident_response.json()["resourceSets"][0]["estimatedTotal"])
                traffic_incidents.append(estimatetotalincidents)
                if estimatetotalincidents > 0:
                    for x in range(estimatetotalincidents):
                        if int(incident_response.json()["resourceSets"][0]["resources"][x]["roadClosed"]) == 1:
                            roadclosures+=1        
                        else:
                            print(0)
                road_closures.append(roadclosures)
                break
            else:
                print("Match did not happen "+business["name"]+" AND "+biz_name)
    else:
        print("DID NOT GET Yelp and Bing Data for : "+biz_name+" at "+str(latitude)+","+str(longitude))
    list_of_tuples = list(zip(zipcode, street_nm, bizname, bizrating, category_name, biz_title, biz_lat, biz_long, bound_box, traffic_incidents,road_closures))
    df = pd.DataFrame(list_of_tuples, columns = ['ZipCode', 'Street Name', 'BusinessName', 'BusinessRating', 'Category Name', 'BusinessTitle', 'BusinessLatitude', 'BusinessLongitude', 'BoundBox', 'TrafficIncidents','RoadClosures'])
    
    #display(df)
    return df

### 4. Loop through the businesses and get the traffic incident and business rating data.
- Ratings from Yelp are used to grade the business and the traffic incidents in the area of the business are retrieved from BING to help determine the congestion.

In [4]:
df_create = 'Y'
create_df=pd.DataFrame()
else_df=pd.DataFrame()

for ind in venues_df.index:
    #print(df_result['Name'][ind], df['Stream'][ind])
    #print(venues_df['Name'][ind])
    if(df_create == 'Y'):
        create_df=get_yelp_bing_data(venues_df['Latitude'][ind],venues_df['Longitude'][ind],venues_df['Name'][ind],venues_df['Category Name'][ind],venues_df['Street'][ind])
        new_df=create_df.copy()
        print("Created Copy")
        df_create='N'
    else:
        else_df=get_yelp_bing_data(venues_df['Latitude'][ind],venues_df['Longitude'][ind],venues_df['Name'][ind],venues_df['Category Name'][ind],venues_df['Street'][ind])
        new_df = new_df.append(else_df)
    print("Key count of dataframe:"+str(len(new_df.index)))
print("Finished Processing")        
display(new_df)

Created Copy
Key count of dataframe:1
Key count of dataframe:2
Key count of dataframe:3
Key count of dataframe:4
Key count of dataframe:5
Key count of dataframe:6
Key count of dataframe:7
Key count of dataframe:8
Key count of dataframe:9
Key count of dataframe:10
Key count of dataframe:11
Key count of dataframe:12
Key count of dataframe:13
Key count of dataframe:14
Key count of dataframe:15
Key count of dataframe:16
Key count of dataframe:17
Key count of dataframe:18
Key count of dataframe:19
Key count of dataframe:20
Key count of dataframe:21
Key count of dataframe:22
Key count of dataframe:23
Key count of dataframe:24
Key count of dataframe:25
Key count of dataframe:26
Key count of dataframe:27
Key count of dataframe:28
Key count of dataframe:29
Key count of dataframe:30
Key count of dataframe:31
Key count of dataframe:32
Key count of dataframe:33
Key count of dataframe:34
Key count of dataframe:35
Key count of dataframe:36
Key count of dataframe:37
Key count of dataframe:38
Key coun

,ZipCode,Street Name,BusinessName,BusinessRating,Category Name,BusinessTitle,BusinessLatitude,BusinessLongitude,BoundBox,TrafficIncidents,RoadClosures
0,95050,El Camino,Poke Poke Fish Bar,4.5,Seafood Restaurant,"Poke , Sushi Bars",37.351913,-121.967506,"37.307,-122.024,37.397,-121.911",16,16
0,95050,El Camino Real,Sprouts Farmers Market,3.5,Grocery Store,"Grocery , Health Markets , Organic Stores",37.350140,-121.962470,"37.305,-122.019,37.395,-121.906",15,15
0,95050,El Camino Real Ste 104105,Dusita Thai Cuisine,4.0,Thai Restaurant,Thai,37.352730,-121.965320,"37.308,-122.022,37.398,-121.909",17,17
0,95053,El Camino Real,Mission Santa Clara de Asis,4.5,Church,"Churches , Landmarks & Historical Buildings",37.349247,-121.941561,"37.304,-121.998,37.394,-121.885",23,23
0,95110,Coleman Ave,Airfield Supply Company,4.0,Marijuana Dispensary,Cannabis Dispensaries,37.356800,-121.931870,"37.312,-121.988,37.402,-121.875",22,22
...,...,...,...,...,...,...,...,...,...,...,...
0,95113,S 1st St,Original Gravity Public House,4.0,Pub,Pubs,37.335056,-121.889605,"37.29,-121.946,37.38,-121.833",20,20
0,94087,Daffodil Ct.,Braly Park,4.0,Park,"Parks , Playgrounds",37.364880,-122.019065,"37.32,-122.076,37.41,-121.962",14,14
0,95110,Pedro Square,Market Beer Company,4.0,Pub,Beer Bar,37.336371,-121.894950,"37.291,-121.952,37.381,-121.838",20,20
0,95131,Murphy Ave Ste A,California Sourdough,4.0,Sandwich Place,"Delis , Sandwiches",37.382313,-121.894814,"37.337,-121.951,37.427,-121.838",17,17


### 5. Map out the businesses.
- Use Folium to output the map

In [5]:
address = NEAR_LOCATION
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of "+address+" are {}, {}.'.format(latitude, longitude))

map_address = folium.Map(location=[latitude, longitude], zoom_start=9)

for lat, lng, ZipCode, BusinessName in zip(new_df['BusinessLatitude'], new_df['BusinessLongitude'], new_df['ZipCode'], new_df['BusinessName']):
    label = '{}, {}'.format(ZipCode, BusinessName)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_address)

map_address

The geograpical coordinate of "+address+" are 37.2333253, -121.6846349.


### 6. Order the data by business rating in descending and get the top 50. Get the zipcode's latitude and longitude. Also get the unique business categories.
- The idea is to get the top 50 rated businesses by rating and then analyse the areas by that.
- Also to store the latitude and longitude of the zipcodes.

In [6]:
#Group by BusinessRating and sort by it. Take top 50 in descending order of BusinessRating.
group_df = new_df.groupby('BusinessRating').apply(lambda x: x).sort_values(by=['BusinessRating'], ascending=False).head(50)
#group_df = new_df.groupby('BusinessRating').apply(lambda x: x).sort_values(by=['BusinessRating'], ascending=False)

display(group_df)
#Insert two dummy columns for the zipcode latitude and longitude
#group_df.insert(1,'Zip_Lat','')
#group_df.insert(2,'Zip_Long','')

print('Got the top 50')

def get_zip_coordinates(zipcode):
    address = zipcode
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of ZIP "+address+" are {}, {}.'.format(latitude, longitude))
    return latitude, longitude

def get_zip_in_loop(dframe):
    ziplat, ziplong = [], []
    for row in group_df.itertuples():
        latude, longtude = get_zip_coordinates(row.ZipCode)
        ziplat.append(latude)
        ziplong.append(longtude)
    return pd.Series({'Zip_Lat': ziplat,'Zip_Long': ziplong})

group_df[['Zip_Lat','Zip_Long']] = get_zip_in_loop(group_df)

#group_df

new_df_orig=new_df.copy(deep=True)

new_df=group_df[['ZipCode','Zip_Lat','Zip_Long','Street Name','Category Name','BusinessName','BusinessRating','TrafficIncidents']]

#new_df

print('Fewer columns got')

#new_df2 = group_df.groupby(['ZipCode', 'Street Name', 'Category Name']).count().reset_index()

new_df2 = new_df.groupby(['ZipCode', 'Street Name', 'Category Name']).agg({'BusinessName':'count', 'TrafficIncidents': 'sum'}).reset_index().rename(columns={'BusinessName':'BusinessCount'})

new_df3=new_df2.copy(deep=True)

new_df4 = new_df.groupby('BusinessRating').apply(lambda x: x).sort_values(by=['ZipCode','BusinessRating'], ascending=True)

#display(new_df4)

print('There are {} uniques business categories.'.format(len(new_df2['Category Name'].unique())))

new_df2['Category Name'].unique()


display(new_df2)

,ZipCode,Street Name,BusinessName,BusinessRating,Category Name,BusinessTitle,BusinessLatitude,BusinessLongitude,BoundBox,TrafficIncidents,RoadClosures
0,94612,Saratoga Ave,Lucky Foot Spa,5.0,Spa,Massage,37.802510,-122.268240,"37.758,-122.325,37.847,-122.211",14,14
0,95054,Woodward Ave,Academy Of Self Defense,5.0,Gym,"Martial Arts , Boot Camps , Cardio Classes",37.384730,-121.945190,"37.34,-122.002,37.43,-121.889",21,21
0,95128,Business Cir,Walia Ethiopian Cuisine,4.5,Ethiopian Restaurant,"Ethiopian , Gluten-Free , Vegetarian",37.322462,-121.932442,"37.277,-121.989,37.367,-121.876",15,15
0,95053,El Camino Real,Mission Santa Clara de Asis,4.5,Church,"Churches , Landmarks & Historical Buildings",37.349247,-121.941561,"37.304,-121.998,37.394,-121.885",23,23
0,95051,Homestead Rd,Stan's Donut Shop,4.5,Donut Shop,Donuts,37.338862,-121.973105,"37.294,-122.03,37.384,-121.917",15,15
0,95050,The Alameda,Ike's Love & Sandwiches,4.5,Sandwich Place,Sandwiches,37.344654,-121.932826,"37.3,-121.989,37.39,-121.876",22,22
0,95054,Space Park Dr,Athena Grill Catering,4.5,Greek Restaurant,Caterers,37.376520,-121.956310,"37.332,-122.013,37.421,-121.9",20,20
0,95126,Ave.,San Jose Municipal Rose Garden,4.5,Garden,"Parks , Botanical Gardens",37.331864,-121.928601,"37.287,-121.985,37.377,-121.872",16,16
0,95117,Stevens Creek Blvd,A Slice of New York,4.5,Pizza Place,Pizza,37.323524,-121.959063,"37.279,-122.016,37.368,-121.903",9,9
0,95126,Naglee Ave,Zanotto's Family Market,4.5,Grocery Store,Grocery,37.327902,-121.931447,"37.283,-121.988,37.373,-121.875",16,16


Got the top 50
The geograpical coordinate of ZIP "+address+" are 37.808733104072935, -122.26807445956773.
The geograpical coordinate of ZIP "+address+" are 37.38900748069918, -121.96825727851963.
The geograpical coordinate of ZIP "+address+" are 37.314141144439276, -121.93506899441226.
The geograpical coordinate of ZIP "+address+" are 44.95018753715243, 34.11970800387856.
The geograpical coordinate of ZIP "+address+" are 37.35258260126035, -121.98728586684243.
The geograpical coordinate of ZIP "+address+" are 48.310427, 18.094748304292857.
The geograpical coordinate of ZIP "+address+" are 37.38900748069918, -121.96825727851963.
The geograpical coordinate of ZIP "+address+" are 50.2032392, 11.935056246665132.
The geograpical coordinate of ZIP "+address+" are 37.31059783205248, -121.9641185670399.
The geograpical coordinate of ZIP "+address+" are 50.2032392, 11.935056246665132.
The geograpical coordinate of ZIP "+address+" are 37.35258260126035, -121.98728586684243.
The geograpical coord

,ZipCode,Street Name,Category Name,BusinessCount,TrafficIncidents
0,94025,N Market St,Caribbean Restaurant,1,4
1,94085,E Duane Ave,Mexican Restaurant,1,12
2,94085,Kern Ave,Sporting Goods Shop,1,15
3,94085,Stewart Dr,Climbing Gym,1,12
4,94086,Reed Ave,Sandwich Place,1,14
5,94087,Daffodil Ct.,Park,1,14
6,94087,E El Camino Real,Mini Golf,1,9
7,94087,E El Camino Real Sunnyvale,Korean Restaurant,1,16
8,94612,Saratoga Ave,Spa,1,14
9,95014,N Tantau Ave,Roof Deck,1,7


### 7. Pivot to have the different categories of businesses and trafficincidents by PostalCode.

- To determine most common business category and also see the trafficincidents by PostalCode.

In [7]:
area_cat_breakup = pd.get_dummies(new_df2[['Category Name']], prefix="", prefix_sep="")
print("all cat keys")
print(area_cat_breakup.keys())
area_cat_breakup['ZipCode'] = new_df2['ZipCode'] 
area_cat_breakup['Street Name'] = new_df2['Street Name']
area_cat_breakup['TrafficIncidents'] = new_df2['TrafficIncidents'] 



# move zipcode, street name column to be at the first
fixed_columns = list(area_cat_breakup.columns[-3:]) + list(area_cat_breakup.columns[:-3])

area_cat_breakup = area_cat_breakup[fixed_columns]

#area_grouped = area_cat_breakup.groupby(["ZipCode", "Street Name"]).mean().reset_index()
area_grouped = area_cat_breakup.groupby(["ZipCode"]).mean().reset_index()

print(area_grouped.shape)
area_grouped



all cat keys
Index(['Asian Restaurant', 'BBQ Joint', 'Bookstore', 'Breakfast Spot',
       'Brewery', 'Café', 'Caribbean Restaurant', 'Church', 'Climbing Gym',
       'Coffee Shop', 'Donut Shop', 'Electronics Store',
       'Ethiopian Restaurant', 'Fast Food Restaurant', 'Food Court', 'Garden',
       'German Restaurant', 'Greek Restaurant', 'Grocery Store', 'Gym',
       'Gym / Fitness Center', 'Hockey Arena', 'Ice Cream Shop',
       'Israeli Restaurant', 'Korean Restaurant', 'Mediterranean Restaurant',
       'Mexican Restaurant', 'Mini Golf', 'Park', 'Pizza Place', 'Plaza',
       'Pub', 'Ramen Restaurant', 'Roof Deck', 'Sandwich Place',
       'Seafood Restaurant', 'Spa', 'Sporting Goods Shop',
       'Theme Park Ride / Attraction', 'Yoga Studio'],
      dtype='object')
(19, 42)


,ZipCode,TrafficIncidents,Asian Restaurant,BBQ Joint,Bookstore,Breakfast Spot,Brewery,Café,Caribbean Restaurant,Church,...,Plaza,Pub,Ramen Restaurant,Roof Deck,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Theme Park Ride / Attraction,Yoga Studio
0,94025,4.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,1.0,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
1,94085,13.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.333333,0.00,0.0
2,94086,14.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.000000,1.000000,0.00,0.0,0.000000,0.00,0.0
3,94087,13.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
4,94612,14.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,1.0,0.000000,0.00,0.0
5,95014,5.666667,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.166667,0.000000,0.00,0.0,0.000000,0.00,0.0
6,95050,17.750000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.25,0.000000,0.250000,0.25,0.0,0.000000,0.00,0.0
7,95051,14.250000,0.000000,0.000000,0.000000,0.00,0.00,0.25,0.0,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
8,95053,23.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,1.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
9,95054,19.500000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.25,0.0


In [8]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
#areaColumns = ['ZipCode', 'Category Name']
areaColumns = ['ZipCode']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
df_new_s = pd.DataFrame(columns=columns)
print('printing venues_sorted first')
display(df_new_s)
df_new_s['ZipCode'] = area_grouped['ZipCode']
display(df_new_s)
display(df_new_s.shape)
display(area_grouped.shape[0])

# add clustering labels and sort
#df_merged=area_grouped.join(area_grouped.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
#df_merged=pd.merge(area_grouped.join(area_grouped, on="PostalCode")
#toronto_merged.sort_values(["Cluster Labels"], inplace=True)

#print(columns)

for ind in np.arange(area_grouped.shape[0]):
    row_categories = area_grouped.iloc[ind, :].iloc[3:]
    print('row_categories')
    print(row_categories)
    row_categories_sorted = row_categories.sort_values(ascending=False)
    print('row_categories_sorted')
    df_new_s.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]
    
print('final')
#print(df_new_s.shape)
display(df_new_s)

printing venues_sorted first


,ZipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


,ZipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,94085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,94086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,94087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,94612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,95014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,95050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,95051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,95053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,95054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(19, 11)

19

row_categories
BBQ Joint                       0.0
Bookstore                       0.0
Breakfast Spot                  0.0
Brewery                         0.0
Café                            0.0
Caribbean Restaurant            1.0
Church                          0.0
Climbing Gym                    0.0
Coffee Shop                     0.0
Donut Shop                      0.0
Electronics Store               0.0
Ethiopian Restaurant            0.0
Fast Food Restaurant            0.0
Food Court                      0.0
Garden                          0.0
German Restaurant               0.0
Greek Restaurant                0.0
Grocery Store                   0.0
Gym                             0.0
Gym / Fitness Center            0.0
Hockey Arena                    0.0
Ice Cream Shop                  0.0
Israeli Restaurant              0.0
Korean Restaurant               0.0
Mediterranean Restaurant        0.0
Mexican Restaurant              0.0
Mini Golf                       0.0
Park         

,ZipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94025,Caribbean Restaurant,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
1,94085,Sporting Goods Shop,Mexican Restaurant,Climbing Gym,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mini Golf,Park
2,94086,Sandwich Place,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
3,94087,Korean Restaurant,Mini Golf,Park,BBQ Joint,Plaza,Israeli Restaurant,Mediterranean Restaurant,Mexican Restaurant,Pizza Place,Pub
4,94612,Spa,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
5,95014,Electronics Store,Roof Deck,Korean Restaurant,Pizza Place,Coffee Shop,Israeli Restaurant,Sandwich Place,Seafood Restaurant,Spa,Ramen Restaurant
6,95050,Ice Cream Shop,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Pizza Place,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf
7,95051,Gym / Fitness Center,Café,Coffee Shop,Donut Shop,Pub,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park
8,95053,Church,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
9,95054,Theme Park Ride / Attraction,Gym,Greek Restaurant,Fast Food Restaurant,BBQ Joint,Pizza Place,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf


### 8. Cluster the areas based on the business category and traffic incidents.

In [9]:
kclusters = 15



area_grouped_clustering = area_grouped.drop(["ZipCode"], 1)
display(area_grouped_clustering)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(area_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(len(kmeans.labels_))

# add clustering labels
df_new_s.insert(0, 'Cluster Labels', kmeans.labels_)

display(df_new_s)

C:\Users\CHETAN~1\AppData\Local\Temp/ipykernel_15340/217474047.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  area_grouped_clustering = area_grouped.drop(["ZipCode"], 1)


,TrafficIncidents,Asian Restaurant,BBQ Joint,Bookstore,Breakfast Spot,Brewery,Café,Caribbean Restaurant,Church,Climbing Gym,...,Plaza,Pub,Ramen Restaurant,Roof Deck,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Theme Park Ride / Attraction,Yoga Studio
0,4.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,1.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
1,13.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.333333,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.333333,0.00,0.0
2,14.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.000000,1.000000,0.00,0.0,0.000000,0.00,0.0
3,13.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
4,14.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,1.0,0.000000,0.00,0.0
5,5.666667,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.166667,0.000000,0.00,0.0,0.000000,0.00,0.0
6,17.750000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.25,0.000000,0.250000,0.25,0.0,0.000000,0.00,0.0
7,14.250000,0.000000,0.000000,0.000000,0.00,0.00,0.25,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
8,23.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,1.0,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0
9,19.500000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.0,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.25,0.0


19


,Cluster Labels,ZipCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,94025,Caribbean Restaurant,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
1,2,94085,Sporting Goods Shop,Mexican Restaurant,Climbing Gym,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mini Golf,Park
2,12,94086,Sandwich Place,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
3,2,94087,Korean Restaurant,Mini Golf,Park,BBQ Joint,Plaza,Israeli Restaurant,Mediterranean Restaurant,Mexican Restaurant,Pizza Place,Pub
4,13,94612,Spa,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
5,9,95014,Electronics Store,Roof Deck,Korean Restaurant,Pizza Place,Coffee Shop,Israeli Restaurant,Sandwich Place,Seafood Restaurant,Spa,Ramen Restaurant
6,14,95050,Ice Cream Shop,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Pizza Place,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf
7,5,95051,Gym / Fitness Center,Café,Coffee Shop,Donut Shop,Pub,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park
8,10,95053,Church,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
9,6,95054,Theme Park Ride / Attraction,Gym,Greek Restaurant,Fast Food Restaurant,BBQ Joint,Pizza Place,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf


### 9. Merge the Data.

In [10]:
# create a merged dataframe
area_merged = new_df_orig.copy()
display(area_merged)
area_merged = area_merged.drop(["Street Name", "Category Name","BusinessName","BusinessRating"], 1)

# add clustering labels and sort
area_merged=area_merged.join(df_new_s.set_index("ZipCode"), on="ZipCode")
area_merged.sort_values(["Cluster Labels"], inplace=True)

print(area_merged.shape)
display(area_merged) 

,ZipCode,Street Name,BusinessName,BusinessRating,Category Name,BusinessTitle,BusinessLatitude,BusinessLongitude,BoundBox,TrafficIncidents,RoadClosures
0,95050,El Camino,Poke Poke Fish Bar,4.5,Seafood Restaurant,"Poke , Sushi Bars",37.351913,-121.967506,"37.307,-122.024,37.397,-121.911",16,16
0,95050,El Camino Real,Sprouts Farmers Market,3.5,Grocery Store,"Grocery , Health Markets , Organic Stores",37.350140,-121.962470,"37.305,-122.019,37.395,-121.906",15,15
0,95050,El Camino Real Ste 104105,Dusita Thai Cuisine,4.0,Thai Restaurant,Thai,37.352730,-121.965320,"37.308,-122.022,37.398,-121.909",17,17
0,95053,El Camino Real,Mission Santa Clara de Asis,4.5,Church,"Churches , Landmarks & Historical Buildings",37.349247,-121.941561,"37.304,-121.998,37.394,-121.885",23,23
0,95110,Coleman Ave,Airfield Supply Company,4.0,Marijuana Dispensary,Cannabis Dispensaries,37.356800,-121.931870,"37.312,-121.988,37.402,-121.875",22,22
...,...,...,...,...,...,...,...,...,...,...,...
0,95113,S 1st St,Original Gravity Public House,4.0,Pub,Pubs,37.335056,-121.889605,"37.29,-121.946,37.38,-121.833",20,20
0,94087,Daffodil Ct.,Braly Park,4.0,Park,"Parks , Playgrounds",37.364880,-122.019065,"37.32,-122.076,37.41,-121.962",14,14
0,95110,Pedro Square,Market Beer Company,4.0,Pub,Beer Bar,37.336371,-121.894950,"37.291,-121.952,37.381,-121.838",20,20
0,95131,Murphy Ave Ste A,California Sourdough,4.0,Sandwich Place,"Delis , Sandwiches",37.382313,-121.894814,"37.337,-121.951,37.427,-121.838",17,17


(96, 18)


C:\Users\CHETAN~1\AppData\Local\Temp/ipykernel_15340/673296464.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  area_merged = area_merged.drop(["Street Name", "Category Name","BusinessName","BusinessRating"], 1)


,ZipCode,BusinessTitle,BusinessLatitude,BusinessLongitude,BoundBox,TrafficIncidents,RoadClosures,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,95113,Pubs,37.335056,-121.889605,"37.29,-121.946,37.38,-121.833",20,20,0.0,Pub,Hockey Arena,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
0,95113,"Stadiums & Arenas , Venues & Event Spaces",37.332753,-121.901245,"37.288,-121.958,37.378,-121.845",17,17,0.0,Pub,Hockey Arena,Plaza,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
0,95129,"Ice Cream & Frozen Yogurt , Coffee & Tea",37.314963,-121.978095,"37.27,-122.035,37.36,-121.922",8,8,1.0,Bookstore,Ice Cream Shop,Grocery Store,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
0,95129,"Bookstores , Gift Shops , Cards & Stationery",37.314636,-121.977709,"37.27,-122.034,37.36,-121.921",8,8,1.0,Bookstore,Ice Cream Shop,Grocery Store,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
0,95129,"Mexican , Food Trucks",37.321964,-121.969840,"37.277,-122.026,37.367,-121.913",8,8,1.0,Bookstore,Ice Cream Shop,Grocery Store,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Park,Pizza Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,95050,Thai,37.352730,-121.965320,"37.308,-122.022,37.398,-121.909",17,17,14.0,Ice Cream Shop,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Pizza Place,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf
0,95050,"Grocery , Health Markets , Organic Stores",37.350140,-121.962470,"37.305,-122.019,37.395,-121.906",15,15,14.0,Ice Cream Shop,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Pizza Place,Israeli Restaurant,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Mini Golf
0,95131,"Mediterranean , Middle Eastern , Salad",37.384500,-121.927330,"37.34,-121.984,37.429,-121.871",21,21,14.0,Mediterranean Restaurant,Sandwich Place,Coffee Shop,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mexican Restaurant,Mini Golf,Park
0,95131,Coffee & Tea,37.381897,-121.895884,"37.337,-121.952,37.427,-121.839",17,17,14.0,Mediterranean Restaurant,Sandwich Place,Coffee Shop,BBQ Joint,Plaza,Israeli Restaurant,Korean Restaurant,Mexican Restaurant,Mini Golf,Park


### 10. Visualizing the resulting clusters

In [11]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(area_merged['BusinessLatitude'], area_merged['BusinessLongitude'], area_merged['ZipCode'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters